In [36]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import warnings
warnings.filterwarnings('ignore')

# Movie Profitability #


## Steps In our process: ##


1. ### Import and clean our data ###
    * Movie Budget data is from https://www.the-numbers.com/movie/budgets/all
    * Market data is from https://finance.yahoo.com/quote/SPY?p=SPY&.tsrc=fin-srch  
      
2. ### Explore data with charts ###
    * Looking at summaries is a great way to get a feel for a data set.
    * We identified profit percentage as a metric to investigate
3. ### Analyze the data ###
    * For this data set we analyzed our data for insights mainly by creating and looking at graphs
4. ### Use Analysis to make recommendations
    * From our graphs we came ups with some concrete recomendation based on budget, compettion, market conditions and season.

In [5]:
df=pickle.load(open("merged_file.pickle","rb"))
tn_movie_budgets=df

Below is a printout of the top ten entries in the database sorted by production budget.  
Change the variable below to show the top ten movies in the data sorted by different columns.  
(All financial numbers are in dollars, and may be recorded in scientific notattion where 4.610445e+08 = $4.610445*10^8$)

In [19]:
# Below is a litst of the types of data we have available
tn_movie_budgets.columns

Index(['id', 'release_date', 'title', 'production_budget', 'domestic_gross',
       'worldwide_gross', 'view_date', 'year', 'percent_return_month',
       'percent_return_rolling', 'market_condition', 'disposable_income',
       'nondurable_goods_expenditure', 'domestic_profit', 'foreign_profit',
       'profit_margin', 'domestic_margin', 'foreign_margin',
       'production_budget_rank', 'budget_size', 'count', 'competition_size',
       'month', 'month_num'],
      dtype='object')

In [ ]:
# Change production_budget to any of the column names printed above to sort by that column
column_to_sort_by='production_budget'

In [18]:
tn_movie_budgets.sort_values(by=[column_to_sort_by], ascending=False).head(10)

,id,release_date,title,production_budget,domestic_gross,worldwide_gross,view_date,year,percent_return_month,percent_return_rolling,...,foreign_profit,profit_margin,domestic_margin,foreign_margin,production_budget_rank,budget_size,count,competition_size,month,month_num
2281,93,2009-09-25,Paranormal Activity,502978.751942,1.206242e+08,2.170443e+08,2009-10-01,2009,6.160683,2.352668,...,9.642016e+07,43051.785333,23881.957778,19169.827556,0.028926,Independent,23,Moderately Low,October,10
708,80,2015-07-10,The Gallows,103012.073015,2.345009e+07,4.291120e+07,2015-08-01,2015,-3.780880,-4.584024,...,1.946111e+07,41556.474000,22664.410000,18892.064000,0.053191,Independent,24,Moderately Low,August,8
3156,10,2004-05-07,Super Size Me,82309.737875,1.459968e+07,2.815475e+07,2004-06-01,2004,-2.868599,-0.694435,...,1.355507e+07,34105.858462,17637.489231,16468.369231,0.024876,Independent,12,Low,June,6
3883,82,2005-08-05,My Date With Drew,1341.496134,2.207871e+05,2.207871e+05,2005-09-01,2005,-1.949503,-0.787107,...,0.000000e+00,16358.272727,16358.272727,0.000000,0.004464,Independent,20,Moderately Low,September,9
1263,57,2007-05-16,Once,172332.578900,1.085219e+07,2.679614e+07,2007-06-01,2007,-2.713499,-2.299230,...,1.594395e+07,15449.087333,6197.238000,9251.849333,0.004484,Independent,14,Low,June,6
3266,75,2004-10-08,Primer,8864.125617,5.378751e+05,1.066134e+06,2004-11-01,2004,2.831574,3.487360,...,5.282588e+05,11927.514286,5968.000000,5959.514286,0.004975,Independent,20,Moderately Low,November,11
3128,8,2004-06-11,Napoleon Dynamite,506521.463847,5.640238e+07,5.840536e+07,2004-07-01,2004,0.243594,-1.312502,...,2.002985e+06,11430.678250,11035.239000,395.439250,0.047264,Independent,15,Low,July,7
3245,31,2004-08-06,Open Water,633151.829809,3.862338e+07,7.030346e+07,2004-09-01,2004,1.710066,1.147551,...,3.168008e+07,11003.728200,6000.176400,5003.551800,0.059701,Independent,16,Moderately Low,September,9
2755,81,2006-09-29,Facing the Giants,118269.014701,1.203781e+07,1.211448e+07,2006-10-01,2006,1.988492,2.796937,...,7.667144e+04,10143.159000,10078.331000,64.828000,0.027451,Independent,26,Moderately Low,October,10
1823,12,2012-01-06,The Devil Inside,1000000.000000,5.326294e+07,1.017595e+08,2012-02-01,2012,2.766000,3.553290,...,4.849655e+07,10075.949000,5226.294500,4849.654500,0.121849,Independent,15,Low,February,2


Below we have a chart that shows us the summary statistics of each of the numeric data columns.

After exploring the data, we decided that we wanted to take a deeper look and come up with some insights about profitability.

# Exploring Profitability with Graphs #  


The first thing we decided to do was constrict our search to only movies from the last ten years, and then just look at the distributions of profitability for each year.


![Distribution Of Movie Profitability](images/profitability_distribution_year.png) 

Just by looking at the distribution we can learn a great deal about our data. It looks like the vast majority of films make less than 

![Competition and Profitability](images/competition.png) 

![Markets and Profitability](images/market.png) 

![Seaonality and Profitability](images/bar_spider.png) 